# KddCup-AnomallyDetection


In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.

import numpy as np
import tensorflow as tf
import csv

    
        

Normalized Dataset saved as fullDataNormalized.csv file

Normalized Trainingset saved as trainingSetNormalized.csv file (Normal data)

Dos:dosSet.csv 
u2r:u2r.csv
r2l:r2l.csv
probe:probe.csv

In [2]:
with open('trainingSetNormalized.csv', 'rb') as f1:
    reader = csv.reader(f1)
    trainList = list(reader)
f1.close()    
print("trainingSetNormalized.csv file opened")
with open('dosSet.csv', 'rb') as f2:
    reader = csv.reader(f2)
    dosList = list(reader) 
f2.close()    
print("dosSet.csv file opened")
with open('u2rSet.csv', 'rb') as f3:
    reader = csv.reader(f3)
    u2rList = list(reader) 
f3.close()        
print("u2rSet.csv file opened")
with open('r2lSet.csv', 'rb') as f4:
    reader = csv.reader(f4)
    r2lList = list(reader)
f4.close()    
print("r2lSet.csv file opened")
with open('probeSet.csv', 'rb') as f5:
    reader = csv.reader(f5)
    probeList = list(reader) 
f5.close()    
print("probeSet.csv file opened")

print("OK!")

trainingSetNormalized.csv file opened
dosSet.csv file opened
u2rSet.csv file opened
r2lSet.csv file opened
probeSet.csv file opened
OK!


In [3]:
def file_len(adress):
    f = open(adress)
    nr_of_lines = sum(1 for line in f)
    f.close()
    return nr_of_lines



In [4]:
trainLen = file_len('trainingSetNormalized.csv')
print("Total length of train data : ", trainLen)

dosLen = file_len('dosSet.csv')
print("Total length of dos data : ", dosLen)

u2rLen = file_len('u2rSet.csv')
print("Total length of u2r data : ", u2rLen)

r2lLen = file_len('r2lSet.csv')
print("Total length of r2l data : ", r2lLen)

probeLen = file_len('probeSet.csv')
print("Total length of probe data : ", probeLen)



('Total length of train data : ', 787953)
('Total length of dos data : ', 247256)
('Total length of u2r data : ', 35)
('Total length of r2l data : ', 36)
('Total length of probe data : ', 13295)


In [5]:
# test with probe
trainingSet = np.empty(shape=[trainLen, 41])
testSet = np.empty(shape=[probeLen, 41]) 

for i in range(trainLen):
    for j in range(41):
        trainingSet[i][j] = trainList[i][j]
print("TrainingSet : ", trainingSet.shape)     
for i in range(trainLen):
    trainingSet[i][19] = 0

for i in range(probeLen):
    for j in range(41):
        testSet[i][j] = probeList[i][j] 
for i in range(probeLen):
    testSet[i][19] = 0
print("TestSet : ", testSet.shape)        
#index 19 is nan because std is 0,,, so in matrix' 19th index element = 0

('TrainingSet : ', (787953, 41))
('TestSet : ', (13295, 41))


# test with dos
trainingSet = np.empty(shape=[trainLen, 41])
testSet = np.empty(shape=[dosLen, 41]) 

for i in range(trainLen):
    for j in range(41):
        trainingSet[i][j] = trainList[i][j]
print("TrainingSet : ", trainingSet.shape)     
for i in range(trainLen):
    trainingSet[i][19] = 0

for i in range(dosLen):
    for j in range(41):
        testSet[i][j] = testList[i][j] 
for i in range(dosLen):
    testSet[i][19] = 0
print("TestSet : ", testSet.shape)        
#index 19 is nan because std is 0,,, so in matrix' 19th index element = 0

In [19]:
# Parameters
learning_rate = 0.01
training_epochs = 20
batch_size = 500
display_step = 1


# Network Parameters
n_input = 41
n_hidden_1 = 39 # 1st layer num features
n_hidden_2 = 37 # 2nd layer num features


# tf Graph input (only pictures)
X = tf.placeholder("float", [None, n_input])

weights = {
    'encoder_h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    'encoder_h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'decoder_h1': tf.Variable(tf.random_normal([n_hidden_2, n_hidden_1])),
    'decoder_h2': tf.Variable(tf.random_normal([n_hidden_1, n_input])),
}
biases = {
    'encoder_b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'encoder_b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'decoder_b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'decoder_b2': tf.Variable(tf.random_normal([n_input])),
}

In [20]:
# Building the encoder
def encoder(x):
    # Encoder Hidden layer with sigmoid activation #1
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, weights['encoder_h1']),biases['encoder_b1']))
    # Decoder Hidden layer with sigmoid activation #2
    layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, weights['encoder_h2']),biases['encoder_b2']))
    return layer_2


# Building the decoder
def decoder(x):
    # Encoder Hidden layer with sigmoid activation #1
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, weights['decoder_h1']),biases['decoder_b1']))
    # Decoder Hidden layer with sigmoid activation #2
    layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, weights['decoder_h2']),biases['decoder_b2']))
    return layer_2

# Construct model
encoder_op = encoder(X)
decoder_op = decoder(encoder_op)

# Prediction
y_pred = decoder_op
# Targets (Labels) are the input data.
y_true = X

# Define loss and optimizer, minimize the squared error
cost = tf.reduce_mean(tf.pow(y_true - y_pred, 2))
optimizer = tf.train.RMSPropOptimizer(learning_rate).minimize(cost)

# Initializing the variables
init = tf.initialize_all_variables()

In [37]:
# Launch the graph
# Using InteractiveSession (more convenient while using Notebooks)
sess = tf.InteractiveSession()
sess.run(init)
display_step = 400
num_steps=4001
total_batch = int(trainingSet.shape[0]/batch_size)
print("Total batch = ",total_batch)
# Training cycle
for step in range(num_steps):
# Pick an offset within the training data, which has been randomized.
# Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (trainingSet.shape[0])
# Generate a minibatch.
    batch_data = trainingSet[offset:(offset + batch_size), :]
    _, c = sess.run([optimizer, cost], feed_dict={X: batch_data})
    if step % display_step == 0:
        print("Step Count :" ,step, "Epoch:", '%04d' % (step+1),"cost=", "{:.9f}".format(c))
        
print("Optimization Finished!")


('Total batch = ', 1575)
('Step Count :', 0, 'Epoch:', '0001', 'cost=', '1.914053321')
('Step Count :', 400, 'Epoch:', '0401', 'cost=', '0.106771439')
('Step Count :', 800, 'Epoch:', '0801', 'cost=', '0.109914325')
('Step Count :', 1200, 'Epoch:', '1201', 'cost=', '0.273107499')
('Step Count :', 1600, 'Epoch:', '1601', 'cost=', '0.268151432')
('Step Count :', 2000, 'Epoch:', '2001', 'cost=', '0.102226719')
('Step Count :', 2400, 'Epoch:', '2401', 'cost=', '0.124305092')
('Step Count :', 2800, 'Epoch:', '2801', 'cost=', '0.610939026')
('Step Count :', 3200, 'Epoch:', '3201', 'cost=', '0.208154112')
('Step Count :', 3600, 'Epoch:', '3601', 'cost=', '0.105424687')
('Step Count :', 4000, 'Epoch:', '4001', 'cost=', '0.108449787')
Optimization Finished!


In [34]:
#encodeDecode = sess.run(y_pred, feed_dict={X: testData[:10000]})
encode_decode = sess.run(
    y_pred, feed_dict={X: testSet[:3000]})

In [23]:
encode_decode.shape

(10000, 41)

In [114]:
testSet[13]

array([ -7.67600000e-02,  -3.77861000e+00,  -2.48153000e+00,
        -5.50310000e-01,  -2.18000000e-03,  -3.33000000e-03,
        -4.98000000e-03,  -3.13500000e-02,  -1.71000000e-03,
        -5.40100000e-02,  -8.38000000e-03,  -1.30924000e+00,
        -6.75000000e-03,  -1.66600000e-02,  -9.16000000e-03,
        -1.19300000e-02,  -1.95300000e-02,  -1.82700000e-02,
        -5.83700000e-02,   0.00000000e+00,  -9.80000000e-04,
        -6.16400000e-02,   6.33930000e-01,   3.08658000e+00,
        -4.93780000e-01,  -4.94700000e-01,  -2.95180000e-01,
        -2.95580000e-01,   5.71720000e-01,  -2.70410000e-01,
        -4.42400000e-01,   9.08900000e-01,   8.56020000e-01,
         7.61810000e-01,  -3.71170000e-01,   4.18069000e+00,
        -3.91840000e-01,  -4.95270000e-01,  -4.93190000e-01,
        -3.06120000e-01,  -3.02600000e-01])

In [115]:
encode_decode[13]

array([  3.00228180e-06,   3.35547193e-05,   1.59903686e-06,
         5.88455350e-06,   4.86902463e-05,   2.98202591e-04,
         7.99740592e-05,   8.51102232e-06,   6.04503963e-04,
         1.03884468e-05,   3.89677698e-05,   7.87762880e-01,
         4.59654511e-05,   5.14870189e-05,   4.54760244e-04,
         4.27797379e-04,   2.76705487e-05,   1.80990290e-04,
         2.28837901e-03,   4.85179153e-05,   7.37562414e-06,
         2.05380256e-05,   8.88946772e-07,   2.17084438e-04,
         5.31000410e-07,   7.56725422e-06,   2.21778741e-04,
         2.21505252e-06,   6.37791634e-01,   2.51313686e-05,
         4.01417338e-07,   5.31951741e-07,   8.93490911e-01,
         7.71922886e-01,   1.66875748e-06,   9.99683619e-01,
         1.10073231e-01,   1.96198494e-06,   6.72947863e-06,
         8.62122700e-03,   4.34382207e-04], dtype=float32)

In [35]:
tempCost = 0
totalCost= 0
step = 0

for i in range(encode_decode.shape[0]):
    tempCost = np.mean(pow(testSet[i] - encode_decode[i], 2))
    if step % 50 == 0 :
        print("STEP :", '%04d' % (step+1),"cost = ", "{:.9f}".format(tempCost))
    step +=1
    totalCost+=tempCost
averageCost=float(totalCost/encode_decode.shape[0])
print("Average cost : ", averageCost)

('STEP :', '0001', 'cost = ', '7.800394351')
('STEP :', '0051', 'cost = ', '7.599238015')
('STEP :', '0101', 'cost = ', '7.593955417')
('STEP :', '0151', 'cost = ', '7.593150545')
('STEP :', '0201', 'cost = ', '1.225110118')
('STEP :', '0251', 'cost = ', '7.631004735')
('STEP :', '0301', 'cost = ', '7.614301281')
('STEP :', '0351', 'cost = ', '1.234198174')
('STEP :', '0401', 'cost = ', '1.252255941')
('STEP :', '0451', 'cost = ', '1.254657137')
('STEP :', '0501', 'cost = ', '1.262540130')
('STEP :', '0551', 'cost = ', '3.109626238')
('STEP :', '0601', 'cost = ', '1.441065535')
('STEP :', '0651', 'cost = ', '2.440202469')
('STEP :', '0701', 'cost = ', '2.526070464')
('STEP :', '0751', 'cost = ', '1.701296291')
('STEP :', '0801', 'cost = ', '2.503368418')
('STEP :', '0851', 'cost = ', '1.385560175')
('STEP :', '0901', 'cost = ', '1.385867292')
('STEP :', '0951', 'cost = ', '1.441275451')
('STEP :', '1001', 'cost = ', '2.585022521')
('STEP :', '1051', 'cost = ', '1.444903710')
('STEP :',

# Training normal data  80% and test other normal data

In [216]:
trainingSet2 = np.empty(shape=[640000, 41])
testSet2 = np.empty(shape=[140000,41])
for i in range(trainingSet2.shape[0]):
    for j in range(41):
        trainingSet2[i][j] = trainingSet[i][j]
print("TrainingSet : ", trainingSet2.shape)     
for i in range(trainingSet2.shape[0]):
    trainingSet[i][19] = 0

for i in range(testSet2.shape[0]):
    for j in range(41):
        testSet2[i][j] = trainingSet[i+640000][j] 
for i in range(testSet2.shape[0]):
    testSet[i][19] = 0
print("TestSet : ", testSet2.shape)        
#index 19 is nan because std is 0,,, so in matrix' 19th index element = 0

('TrainingSet : ', (640000, 41))
('TestSet : ', (140000, 41))


In [217]:
# Parameters
learning_rate = 0.01
training_epochs = 20
batch_size = 200
display_step = 1


# Network Parameters
n_input = 41
n_hidden_1 = 40 # 1st layer num features
n_hidden_2 = 35 # 2nd layer num features


# tf Graph input (only pictures)
X = tf.placeholder("float", [None, n_input])

weights = {
    'encoder_h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    'encoder_h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'decoder_h1': tf.Variable(tf.random_normal([n_hidden_2, n_hidden_1])),
    'decoder_h2': tf.Variable(tf.random_normal([n_hidden_1, n_input])),
}
biases = {
    'encoder_b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'encoder_b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'decoder_b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'decoder_b2': tf.Variable(tf.random_normal([n_input])),
}

In [218]:
# Building the encoder
def encoder(x):
    # Encoder Hidden layer with sigmoid activation #1
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, weights['encoder_h1']),biases['encoder_b1']))
    # Decoder Hidden layer with sigmoid activation #2
    layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, weights['encoder_h2']),biases['encoder_b2']))
    return layer_2


# Building the decoder
def decoder(x):
    # Encoder Hidden layer with sigmoid activation #1
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, weights['decoder_h1']),biases['decoder_b1']))
    # Decoder Hidden layer with sigmoid activation #2
    layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, weights['decoder_h2']),biases['decoder_b2']))
    return layer_2

# Construct model
encoder_op = encoder(X)
decoder_op = decoder(encoder_op)

# Prediction
y_pred = decoder_op
# Targets (Labels) are the input data.
y_true = X

# Define loss and optimizer, minimize the squared error
cost = tf.reduce_mean(tf.pow(y_true - y_pred, 2))
optimizer = tf.train.RMSPropOptimizer(learning_rate).minimize(cost)

# Initializing the variables
init = tf.initialize_all_variables()

In [233]:
# Launch the graph
# Using InteractiveSession (more convenient while using Notebooks)
sess = tf.InteractiveSession()
sess.run(init)
display_step = 400
num_steps=4001
total_batch = int(trainingSet2.shape[0]/batch_size)
print("Total batch = ",total_batch)
# Training cycle
for step in range(num_steps):
# Pick an offset within the training data, which has been randomized.
# Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (trainingSet2.shape[0])
# Generate a minibatch.
    batch_data = trainingSet[offset:(offset + batch_size), :]
    _, c = sess.run([optimizer, cost], feed_dict={X: batch_data})
    if step % display_step == 0:
        print("Epoch:", '%04d' % (step+1),"cost=", "{:.9f}".format(c))
        print("Step Count :" ,step);
        
print("Optimization Finished!")


('Total batch = ', 3200)
('Epoch:', '0001', 'cost=', '2.091668367')
('Step Count :', 0)
('Epoch:', '0401', 'cost=', '0.932155490')
('Step Count :', 400)
('Epoch:', '0801', 'cost=', '0.104474127')
('Step Count :', 800)
('Epoch:', '1201', 'cost=', '0.089258589')
('Step Count :', 1200)
('Epoch:', '1601', 'cost=', '0.094946451')
('Step Count :', 1600)
('Epoch:', '2001', 'cost=', '0.106285311')
('Step Count :', 2000)
('Epoch:', '2401', 'cost=', '0.812773526')
('Step Count :', 2400)
('Epoch:', '2801', 'cost=', '0.095268220')
('Step Count :', 2800)
('Epoch:', '3201', 'cost=', '1.068478584')
('Step Count :', 3200)
('Epoch:', '3601', 'cost=', '0.926778436')
('Step Count :', 3600)
('Epoch:', '4001', 'cost=', '0.103599392')
('Step Count :', 4000)
Optimization Finished!


In [245]:
#encodeDecode = sess.run(y_pred, feed_dict={X: testData[:10000]})
encode_decode = sess.run(
    y_pred, feed_dict={X: testSet2[:100]})

In [244]:
tempCost = 0
totalCost= 0
step = 0

for i in range(encode_decode.shape[0]):
    tempCost = np.mean(pow(testSet2[i] - encode_decode[i], 2))
    print("STEP :", '%04d' % (step+1),"cost = ", "{:.9f}".format(tempCost))
    step +=1
    totalCost+=tempCost
averageCost=float(totalCost/encode_decode.shape[0])
print("Average cost : ", averageCost)

('STEP :', '0001', 'cost = ', '0.278274276')
('STEP :', '0002', 'cost = ', '1.893928860')
('STEP :', '0003', 'cost = ', '0.494011689')
('STEP :', '0004', 'cost = ', '0.148261872')
('STEP :', '0005', 'cost = ', '0.148752706')
('STEP :', '0006', 'cost = ', '0.137089552')
('STEP :', '0007', 'cost = ', '0.114114417')
('STEP :', '0008', 'cost = ', '0.143503141')
('STEP :', '0009', 'cost = ', '0.118217732')
('STEP :', '0010', 'cost = ', '0.291572173')
('STEP :', '0011', 'cost = ', '0.146638538')
('STEP :', '0012', 'cost = ', '0.264558499')
('STEP :', '0013', 'cost = ', '0.221340076')
('STEP :', '0014', 'cost = ', '0.262439112')
('STEP :', '0015', 'cost = ', '0.418259419')
('STEP :', '0016', 'cost = ', '0.323063670')
('STEP :', '0017', 'cost = ', '0.178475629')
('STEP :', '0018', 'cost = ', '0.128138104')
('STEP :', '0019', 'cost = ', '0.134167231')
('STEP :', '0020', 'cost = ', '1.993381003')
('STEP :', '0021', 'cost = ', '0.428362813')
('STEP :', '0022', 'cost = ', '0.111898813')
('STEP :',

#  Train with normal data's 80 % and find anomally

In [95]:
tempSet = trainingSet
np.random.shuffle(tempSet)

In [96]:
trainShuffledData = tempSet[:640000,:]
testShuffled = tempSet[640000:780000,:]


In [97]:
print(" Training data : ", trainShuffledData.shape)
print(" Test data     : ", testShuffled.shape )

(' Training data : ', (640000, 41))
(' Test data     : ', (140000, 41))


In [98]:
# Parameters
learning_rate = 0.01
training_epochs = 20
batch_size = 250
display_step = 1


# Network Parameters
n_input = 41
n_hidden_1 = 38 # 1st layer num features
n_hidden_2 = 35 # 2nd layer num features


# tf Graph input (only pictures)
X = tf.placeholder("float", [None, n_input])

weights = {
    'encoder_h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    'encoder_h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'decoder_h1': tf.Variable(tf.random_normal([n_hidden_2, n_hidden_1])),
    'decoder_h2': tf.Variable(tf.random_normal([n_hidden_1, n_input])),
}
biases = {
    'encoder_b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'encoder_b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'decoder_b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'decoder_b2': tf.Variable(tf.random_normal([n_input])),
}


# Building the encoder
def encoder(x):
    # Encoder Hidden layer with sigmoid activation #1
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, weights['encoder_h1']),biases['encoder_b1']))
    # Decoder Hidden layer with sigmoid activation #2
    layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, weights['encoder_h2']),biases['encoder_b2']))
    return layer_2


# Building the decoder
def decoder(x):
    # Encoder Hidden layer with sigmoid activation #1
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, weights['decoder_h1']),biases['decoder_b1']))
    # Decoder Hidden layer with sigmoid activation #2
    layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, weights['decoder_h2']),biases['decoder_b2']))
    return layer_2

# Construct model
encoder_op = encoder(X)
decoder_op = decoder(encoder_op)

# Prediction
y_pred = decoder_op
# Targets (Labels) are the input data.
y_true = X

# Define loss and optimizer, minimize the squared error
cost = tf.reduce_mean(tf.pow(y_true - y_pred, 2))
optimizer = tf.train.RMSPropOptimizer(learning_rate).minimize(cost)

# Initializing the variables
init = tf.initialize_all_variables()


# Launch the graph
# Using InteractiveSession (more convenient while using Notebooks)
sess = tf.InteractiveSession()
sess.run(init)
display_step = 400
num_steps=4001
total_batch = int(trainShuffledData.shape[0]/batch_size)
print("Total batch = ",total_batch)
# Training cycle
for step in range(num_steps):
# Pick an offset within the training data, which has been randomized.
# Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (trainShuffledData.shape[0])
# Generate a minibatch.
    batch_data = trainShuffledData[offset:(offset + batch_size), :]
    _, c = sess.run([optimizer, cost], feed_dict={X: batch_data})
    if step % display_step == 0:
        print("Epoch:", '%04d' % (step+1),"cost=", "{:.9f}".format(c))
        print("Step Count :" ,step);
        
print("Optimization Finished!")




('Total batch = ', 2560)
('Epoch:', '0001', 'cost=', '0.845439613')
('Step Count :', 0)
('Epoch:', '0401', 'cost=', '0.319008082')
('Step Count :', 400)
('Epoch:', '0801', 'cost=', '0.592409730')
('Step Count :', 800)
('Epoch:', '1201', 'cost=', '0.633674324')
('Step Count :', 1200)
('Epoch:', '1601', 'cost=', '0.385230154')
('Step Count :', 1600)
('Epoch:', '2001', 'cost=', '0.675249636')
('Step Count :', 2000)
('Epoch:', '2401', 'cost=', '0.324178785')
('Step Count :', 2400)
('Epoch:', '2801', 'cost=', '0.807040274')
('Step Count :', 2800)
('Epoch:', '3201', 'cost=', '0.300024778')
('Step Count :', 3200)
('Epoch:', '3601', 'cost=', '0.923783362')
('Step Count :', 3600)
('Epoch:', '4001', 'cost=', '0.377356678')
('Step Count :', 4000)
Optimization Finished!


In [105]:
anomally =[]
anomalNumber = 0

#encodeDecode = sess.run(y_pred, feed_dict={X: testData[:10000]})
encode_decode = sess.run(
    y_pred, feed_dict={X: testShuffled})

tempCost = 0
totalCost= 0
step = 0

for i in range(encode_decode.shape[0]):
    tempCost = np.mean(pow(testShuffled[i] - encode_decode[i], 2))
    if tempCost > 0.5 :
        anomally.append(i)
        anomalNumber += 1
    #print("STEP :", '%04d' % (step+1),"cost = ", "{:.9f}".format(tempCost))
    step +=1
    totalCost+=tempCost
averageCost=float(totalCost/encode_decode.shape[0])
print("Average cost : ", averageCost)

#for i in range(anomalNumber):
#    print("Anomaly data index is : ", anomally[i] )

print("Testing for normal data ", anomalNumber , " anomaly detected ")
correct =  (100 * float(anomalNumber) / float(testShuffled.shape[0]))
print("Number of detected Anomaly ", anomalNumber)
print("Number of total anomaly data ", testShuffled.shape[0])
print("anomally detection for normal data  percenteges is  ", correct)
correct = 100 - correct
print("Correct prediction is ", correct)

('Average cost : ', 0.5258665931149764)
('Testing for normal data ', 16479, ' anomaly detected ')
('Number of detected Anomaly ', 16479)
('Number of total anomaly data ', 140000)
('anomally detection for normal data  percenteges is  ', 11.770714285714286)
('Correct prediction is ', 88.22928571428571)


# Training with normal data and detection anomaly by Dos test

In [74]:
tempSet = trainingSet
np.random.shuffle(tempSet)
trainShuffledData = tempSet
dosTest = np.empty(shape=[dosLen, 41])
for i in range(dosLen):
    for j in range(41):
        dosTest[i][j] = dosList[i][j] 
dosTest[:,19] = 0        
print(" Training data : ", trainShuffledData.shape)
print(" Test data     : ", dosTest.shape )



(' Training data : ', (787953, 41))
(' Test data     : ', (247256, 41))


In [75]:
# Parameters
learning_rate = 0.01
training_epochs = 20
batch_size = 250
display_step = 1


# Network Parameters
n_input = 41
n_hidden_1 = 38 # 1st layer num features
n_hidden_2 = 35 # 2nd layer num features


# tf Graph input (only pictures)
X = tf.placeholder("float", [None, n_input])

weights = {
    'encoder_h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    'encoder_h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'decoder_h1': tf.Variable(tf.random_normal([n_hidden_2, n_hidden_1])),
    'decoder_h2': tf.Variable(tf.random_normal([n_hidden_1, n_input])),
}
biases = {
    'encoder_b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'encoder_b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'decoder_b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'decoder_b2': tf.Variable(tf.random_normal([n_input])),
}


# Building the encoder
def encoder(x):
    # Encoder Hidden layer with sigmoid activation #1
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, weights['encoder_h1']),biases['encoder_b1']))
    # Decoder Hidden layer with sigmoid activation #2
    layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, weights['encoder_h2']),biases['encoder_b2']))
    return layer_2


# Building the decoder
def decoder(x):
    # Encoder Hidden layer with sigmoid activation #1
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, weights['decoder_h1']),biases['decoder_b1']))
    # Decoder Hidden layer with sigmoid activation #2
    layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, weights['decoder_h2']),biases['decoder_b2']))
    return layer_2

# Construct model
encoder_op = encoder(X)
decoder_op = decoder(encoder_op)

# Prediction
y_pred = decoder_op
# Targets (Labels) are the input data.
y_true = X

# Define loss and optimizer, minimize the squared error
cost = tf.reduce_mean(tf.pow(y_true - y_pred, 2))
optimizer = tf.train.RMSPropOptimizer(learning_rate).minimize(cost)

# Initializing the variables
init = tf.initialize_all_variables()


# Launch the graph
# Using InteractiveSession (more convenient while using Notebooks)
sess = tf.InteractiveSession()
sess.run(init)
display_step = 400
num_steps=4001
total_batch = int(trainShuffledData.shape[0]/batch_size)
print("Total batch = ",total_batch)
# Training cycle
for step in range(num_steps):
# Pick an offset within the training data, which has been randomized.
# Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (trainShuffledData.shape[0])
# Generate a minibatch.
    batch_data = trainShuffledData[offset:(offset + batch_size), :]
    _, c = sess.run([optimizer, cost], feed_dict={X: batch_data})
    if step % display_step == 0:
        print("Epoch:", '%04d' % (step+1),"cost=", "{:.9f}".format(c))
        print("Step Count :" ,step);
        
print("Optimization Finished!")




('Total batch = ', 3151)
('Epoch:', '0001', 'cost=', '1.013197064')
('Step Count :', 0)
('Epoch:', '0401', 'cost=', '0.309251010')
('Step Count :', 400)
('Epoch:', '0801', 'cost=', '0.274771899')
('Step Count :', 800)
('Epoch:', '1201', 'cost=', '1.446210027')
('Step Count :', 1200)
('Epoch:', '1601', 'cost=', '0.367053807')
('Step Count :', 1600)
('Epoch:', '2001', 'cost=', '4.279848099')
('Step Count :', 2000)
('Epoch:', '2401', 'cost=', '0.764706135')
('Step Count :', 2400)
('Epoch:', '2801', 'cost=', '0.423874378')
('Step Count :', 2800)
('Epoch:', '3201', 'cost=', '0.614094377')
('Step Count :', 3200)
('Epoch:', '3601', 'cost=', '0.327837259')
('Step Count :', 3600)
('Epoch:', '4001', 'cost=', '0.478001237')
('Step Count :', 4000)
Optimization Finished!


In [80]:
anomally =[]
anomalNumber = 0
#encodeDecode = sess.run(y_pred, feed_dict={X: testData[:10000]})
encode_decode = sess.run(
    y_pred, feed_dict={X: dosTest})

tempCost = 0
totalCost= 0
step = 0

for i in range(encode_decode.shape[0]):
    tempCost = np.mean(pow(dosTest[i] - encode_decode[i], 2))
    if tempCost > 0.5 :
        anomally.append(i)
        anomalNumber += 1
    #print("STEP :", '%04d' % (step+1),"cost = ", "{:.9f}".format(tempCost))
    step +=1
    totalCost+=tempCost
averageCost=float(totalCost/encode_decode.shape[0])
print("Average cost : ", averageCost)

#for i in range(anomalNumber):
#    print("Anomaly data index is : ", anomally[i] )

print("Testing for dos data ", anomalNumber , " anomal data detected ")
correct =  (100 * float(anomalNumber) / float(dosLen))
print("Number of detected Anomaly ", anomalNumber)
print("Number of total anomaly data ", dosLen)
print("anomally detection correction for dos data  percenteges is  ", correct)

('Average cost : ', 1.1068676201798393)
('Testing for dos data ', 246291, ' anomal data detected ')
('Number of detected Anomaly ', 246291)
('Number of total anomaly data ', 247256)
('anomally detection correction for dos data  percenteges is  ', 99.60971624551073)


# Training with normal data and detection anomaly by Probe test


In [50]:
tempSet = trainingSet
np.random.shuffle(tempSet)
trainShuffledData = tempSet
probeTest = np.empty(shape=[probeLen, 41])
for i in range(probeLen):
    for j in range(41):
        probeTest[i][j] = probeList[i][j] 
        
print(" Training data : ", trainShuffledData.shape)
print(" Test data     : ", probeTest.shape )



(' Training data : ', (787953, 41))
(' Test data     : ', (13295, 41))


In [71]:
# Parameters
learning_rate = 0.01
training_epochs = 20
batch_size = 250
display_step = 1


# Network Parameters
n_input = 41
n_hidden_1 = 38 # 1st layer num features
n_hidden_2 = 35 # 2nd layer num features


# tf Graph input (only pictures)
X = tf.placeholder("float", [None, n_input])

weights = {
    'encoder_h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    'encoder_h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'decoder_h1': tf.Variable(tf.random_normal([n_hidden_2, n_hidden_1])),
    'decoder_h2': tf.Variable(tf.random_normal([n_hidden_1, n_input])),
}
biases = {
    'encoder_b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'encoder_b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'decoder_b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'decoder_b2': tf.Variable(tf.random_normal([n_input])),
}


# Building the encoder
def encoder(x):
    # Encoder Hidden layer with sigmoid activation #1
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, weights['encoder_h1']),biases['encoder_b1']))
    # Decoder Hidden layer with sigmoid activation #2
    layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, weights['encoder_h2']),biases['encoder_b2']))
    return layer_2


# Building the decoder
def decoder(x):
    # Encoder Hidden layer with sigmoid activation #1
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, weights['decoder_h1']),biases['decoder_b1']))
    # Decoder Hidden layer with sigmoid activation #2
    layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, weights['decoder_h2']),biases['decoder_b2']))
    return layer_2

# Construct model
encoder_op = encoder(X)
decoder_op = decoder(encoder_op)

# Prediction
y_pred = decoder_op
# Targets (Labels) are the input data.
y_true = X

# Define loss and optimizer, minimize the squared error
cost = tf.reduce_mean(tf.pow(y_true - y_pred, 2))
optimizer = tf.train.RMSPropOptimizer(learning_rate).minimize(cost)

# Initializing the variables
init = tf.initialize_all_variables()


# Launch the graph
# Using InteractiveSession (more convenient while using Notebooks)
sess = tf.InteractiveSession()
sess.run(init)
display_step = 400
num_steps=4001
total_batch = int(trainShuffledData.shape[0]/batch_size)
print("Total batch = ",total_batch)
# Training cycle
for step in range(num_steps):
# Pick an offset within the training data, which has been randomized.
# Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (trainShuffledData.shape[0])
# Generate a minibatch.
    batch_data = trainShuffledData[offset:(offset + batch_size), :]
    _, c = sess.run([optimizer, cost], feed_dict={X: batch_data})
    if step % display_step == 0:
        print("Epoch:", '%04d' % (step+1),"cost=", "{:.9f}".format(c))
        print("Step Count :" ,step);
        
print("Optimization Finished!")




('Total batch = ', 3151)
('Epoch:', '0001', 'cost=', '0.972607076')
('Step Count :', 0)
('Epoch:', '0401', 'cost=', '0.291858435')
('Step Count :', 400)
('Epoch:', '0801', 'cost=', '0.291937917')
('Step Count :', 800)
('Epoch:', '1201', 'cost=', '0.226983249')
('Step Count :', 1200)
('Epoch:', '1601', 'cost=', '0.218233779')
('Step Count :', 1600)
('Epoch:', '2001', 'cost=', '0.245217010')
('Step Count :', 2000)
('Epoch:', '2401', 'cost=', '4.505098820')
('Step Count :', 2400)
('Epoch:', '2801', 'cost=', '0.436607093')
('Step Count :', 2800)
('Epoch:', '3201', 'cost=', '0.227027893')
('Step Count :', 3200)
('Epoch:', '3601', 'cost=', '0.305429220')
('Step Count :', 3600)
('Epoch:', '4001', 'cost=', '0.249904200')
('Step Count :', 4000)
Optimization Finished!


In [73]:
anomally =[]
anomalNumber = 0

#encodeDecode = sess.run(y_pred, feed_dict={X: testData[:10000]})
encode_decode = sess.run(
    y_pred, feed_dict={X: probeTest})

tempCost = 0
totalCost= 0
step = 0

for i in range(encode_decode.shape[0]):
    tempCost = np.mean(pow(probeTest[i] - encode_decode[i], 2))
    if tempCost > 0.4 :
        anomally.append(i)
        anomalNumber += 1
    #print("STEP :", '%04d' % (step+1),"cost = ", "{:.9f}".format(tempCost))
    step +=1
    totalCost+=tempCost
averageCost=float(totalCost/encode_decode.shape[0])
print("Average cost : ", averageCost)

#for i in range(anomalNumber):
#    print("Anomaly data index is : ", anomally[i] )

print("Testing for probe data ", anomalNumber , " anomal data detected ")
correct =  (100 * float(anomalNumber) / float(probeTest.shape[0]))
print("Number of detected Anomaly ", anomalNumber)
print("Number of total anomaly data ", probeLen)
print("anomally detection correction for dos data  percenteges is  ", correct)


('Average cost : ', 7.125895367296353)
('Testing for probe data ', 13106, ' anomal data detected ')
('Number of detected Anomaly ', 13106)
('Number of total anomaly data ', 13295)
('anomally detection correction for dos data  percenteges is  ', 98.57841293719443)


# Training with normal data and detection anomaly by u2r test

In [84]:
tempSet = trainingSet
np.random.shuffle(tempSet)
trainShuffledData = tempSet
u2rTest = np.empty(shape=[u2rLen, 41])
for i in range(u2rLen):
    for j in range(41):
        u2rTest[i][j] = u2rList[i][j] 
u2rTest[:,19] = 0        
print(" Training data : ", trainShuffledData.shape)
print(" Test data     : ", u2rTest.shape )


(' Training data : ', (787953, 41))
(' Test data     : ', (35, 41))


In [85]:
# Parameters
learning_rate = 0.01
training_epochs = 20
batch_size = 250
display_step = 1


# Network Parameters
n_input = 41
n_hidden_1 = 38 # 1st layer num features
n_hidden_2 = 35 # 2nd layer num features


# tf Graph input (only pictures)
X = tf.placeholder("float", [None, n_input])

weights = {
    'encoder_h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    'encoder_h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'decoder_h1': tf.Variable(tf.random_normal([n_hidden_2, n_hidden_1])),
    'decoder_h2': tf.Variable(tf.random_normal([n_hidden_1, n_input])),
}
biases = {
    'encoder_b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'encoder_b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'decoder_b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'decoder_b2': tf.Variable(tf.random_normal([n_input])),
}


# Building the encoder
def encoder(x):
    # Encoder Hidden layer with sigmoid activation #1
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, weights['encoder_h1']),biases['encoder_b1']))
    # Decoder Hidden layer with sigmoid activation #2
    layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, weights['encoder_h2']),biases['encoder_b2']))
    return layer_2


# Building the decoder
def decoder(x):
    # Encoder Hidden layer with sigmoid activation #1
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, weights['decoder_h1']),biases['decoder_b1']))
    # Decoder Hidden layer with sigmoid activation #2
    layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, weights['decoder_h2']),biases['decoder_b2']))
    return layer_2

# Construct model
encoder_op = encoder(X)
decoder_op = decoder(encoder_op)

# Prediction
y_pred = decoder_op
# Targets (Labels) are the input data.
y_true = X

# Define loss and optimizer, minimize the squared error
cost = tf.reduce_mean(tf.pow(y_true - y_pred, 2))
optimizer = tf.train.RMSPropOptimizer(learning_rate).minimize(cost)

# Initializing the variables
init = tf.initialize_all_variables()


# Launch the graph
# Using InteractiveSession (more convenient while using Notebooks)
sess = tf.InteractiveSession()
sess.run(init)
display_step = 400
num_steps=4001
total_batch = int(trainShuffledData.shape[0]/batch_size)
print("Total batch = ",total_batch)
# Training cycle
for step in range(num_steps):
# Pick an offset within the training data, which has been randomized.
# Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (trainShuffledData.shape[0])
# Generate a minibatch.
    batch_data = trainShuffledData[offset:(offset + batch_size), :]
    _, c = sess.run([optimizer, cost], feed_dict={X: batch_data})
    if step % display_step == 0:
        print("Epoch:", '%04d' % (step+1),"cost=", "{:.9f}".format(c))
        print("Step Count :" ,step);
        
print("Optimization Finished!")




('Total batch = ', 3151)
('Epoch:', '0001', 'cost=', '1.382441759')
('Step Count :', 0)
('Epoch:', '0401', 'cost=', '0.297699720')
('Step Count :', 400)
('Epoch:', '0801', 'cost=', '0.311143458')
('Step Count :', 800)
('Epoch:', '1201', 'cost=', '0.314602822')
('Step Count :', 1200)
('Epoch:', '1601', 'cost=', '0.832315147')
('Step Count :', 1600)
('Epoch:', '2001', 'cost=', '0.346339673')
('Step Count :', 2000)
('Epoch:', '2401', 'cost=', '0.544762552')
('Step Count :', 2400)
('Epoch:', '2801', 'cost=', '0.361529738')
('Step Count :', 2800)
('Epoch:', '3201', 'cost=', '0.668155849')
('Step Count :', 3200)
('Epoch:', '3601', 'cost=', '0.295091957')
('Step Count :', 3600)
('Epoch:', '4001', 'cost=', '0.203608111')
('Step Count :', 4000)
Optimization Finished!


In [87]:
anomally =[]
anomalNumber = 0

#encodeDecode = sess.run(y_pred, feed_dict={X: testData[:10000]})
encode_decode = sess.run(
    y_pred, feed_dict={X: u2rTest})

tempCost = 0
totalCost= 0
step = 0

for i in range(encode_decode.shape[0]):
    tempCost = np.mean(pow(u2rTest[i] - encode_decode[i], 2))
    if tempCost > 0.4 :
        anomally.append(i)
        anomalNumber += 1
    #print("STEP :", '%04d' % (step+1),"cost = ", "{:.9f}".format(tempCost))
    step +=1
    totalCost+=tempCost
averageCost=float(totalCost/encode_decode.shape[0])
print("Average cost : ", averageCost)

#for i in range(anomalNumber):
#    print("Anomaly data index is : ", anomally[i] )

print("Testing for u2r data ", anomalNumber , " anomal data detected ")
correct =  (100 * float(anomalNumber) / float(u2rLen))
print("Number of detected Anomaly ", anomalNumber)
print("Number of total anomaly data ", u2rLen)
print("anomally detection correction for u2r data  percenteges is  ", correct)


('Average cost : ', 48.00187496361766)
('Testing for u2r data ', 32, ' anomal data detected ')
('Number of detected Anomaly ', 32)
('Number of total anomaly data ', 35)
('anomally detection correction for u2r data  percenteges is  ', 91.42857142857143)


# Training with normal data and detection anomaly by r2l test

In [88]:
tempSet = trainingSet
np.random.shuffle(tempSet)
trainShuffledData = tempSet
r2lTest = np.empty(shape=[r2lLen, 41])
for i in range(r2lLen):
    for j in range(41):
        r2lTest[i][j] = r2lList[i][j] 
r2lTest[:,19] = 0        
print(" Training data : ", trainShuffledData.shape)
print(" Test data     : ", r2lTest.shape )


(' Training data : ', (787953, 41))
(' Test data     : ', (36, 41))


In [89]:
# Parameters
learning_rate = 0.01
training_epochs = 20
batch_size = 250
display_step = 1


# Network Parameters
n_input = 41
n_hidden_1 = 38 # 1st layer num features
n_hidden_2 = 35 # 2nd layer num features


# tf Graph input (only pictures)
X = tf.placeholder("float", [None, n_input])

weights = {
    'encoder_h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    'encoder_h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'decoder_h1': tf.Variable(tf.random_normal([n_hidden_2, n_hidden_1])),
    'decoder_h2': tf.Variable(tf.random_normal([n_hidden_1, n_input])),
}
biases = {
    'encoder_b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'encoder_b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'decoder_b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'decoder_b2': tf.Variable(tf.random_normal([n_input])),
}


# Building the encoder
def encoder(x):
    # Encoder Hidden layer with sigmoid activation #1
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, weights['encoder_h1']),biases['encoder_b1']))
    # Decoder Hidden layer with sigmoid activation #2
    layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, weights['encoder_h2']),biases['encoder_b2']))
    return layer_2


# Building the decoder
def decoder(x):
    # Encoder Hidden layer with sigmoid activation #1
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, weights['decoder_h1']),biases['decoder_b1']))
    # Decoder Hidden layer with sigmoid activation #2
    layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, weights['decoder_h2']),biases['decoder_b2']))
    return layer_2

# Construct model
encoder_op = encoder(X)
decoder_op = decoder(encoder_op)

# Prediction
y_pred = decoder_op
# Targets (Labels) are the input data.
y_true = X

# Define loss and optimizer, minimize the squared error
cost = tf.reduce_mean(tf.pow(y_true - y_pred, 2))
optimizer = tf.train.RMSPropOptimizer(learning_rate).minimize(cost)

# Initializing the variables
init = tf.initialize_all_variables()


# Launch the graph
# Using InteractiveSession (more convenient while using Notebooks)
sess = tf.InteractiveSession()
sess.run(init)
display_step = 400
num_steps=4001
total_batch = int(trainShuffledData.shape[0]/batch_size)
print("Total batch = ",total_batch)
# Training cycle
for step in range(num_steps):
# Pick an offset within the training data, which has been randomized.
# Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (trainShuffledData.shape[0])
# Generate a minibatch.
    batch_data = trainShuffledData[offset:(offset + batch_size), :]
    _, c = sess.run([optimizer, cost], feed_dict={X: batch_data})
    if step % display_step == 0:
        print("Epoch:", '%04d' % (step+1),"cost=", "{:.9f}".format(c))
        print("Step Count :" ,step);
        
print("Optimization Finished!")




('Total batch = ', 3151)
('Epoch:', '0001', 'cost=', '0.961491048')
('Step Count :', 0)
('Epoch:', '0401', 'cost=', '0.251833797')
('Step Count :', 400)
('Epoch:', '0801', 'cost=', '0.496878624')
('Step Count :', 800)
('Epoch:', '1201', 'cost=', '0.281488627')
('Step Count :', 1200)
('Epoch:', '1601', 'cost=', '0.574708700')
('Step Count :', 1600)
('Epoch:', '2001', 'cost=', '0.304099053')
('Step Count :', 2000)
('Epoch:', '2401', 'cost=', '0.330498070')
('Step Count :', 2400)
('Epoch:', '2801', 'cost=', '0.503193319')
('Step Count :', 2800)
('Epoch:', '3201', 'cost=', '0.227206394')
('Step Count :', 3200)
('Epoch:', '3601', 'cost=', '0.225252435')
('Step Count :', 3600)
('Epoch:', '4001', 'cost=', '0.364628971')
('Step Count :', 4000)
Optimization Finished!


In [94]:
anomally =[]
anomalNumber = 0

#encodeDecode = sess.run(y_pred, feed_dict={X: testData[:10000]})
encode_decode = sess.run(
    y_pred, feed_dict={X: r2lTest})

tempCost = 0
totalCost= 0
step = 0

for i in range(encode_decode.shape[0]):
    tempCost = np.mean(pow(r2lTest[i] - encode_decode[i], 2))
    if tempCost > 0.5 :
        anomally.append(i)
        anomalNumber += 1
    #print("STEP :", '%04d' % (step+1),"cost = ", "{:.9f}".format(tempCost))
    step +=1
    totalCost+=tempCost
averageCost=float(totalCost/encode_decode.shape[0])
print("Average cost : ", averageCost)

#for i in range(anomalNumber):
#    print("Anomaly data index is : ", anomally[i] )

print("Testing for r2l data ", anomalNumber , " anomal data detected ")
correct =  (100 * float(anomalNumber) / float(u2rLen))
print("Number of detected Anomaly ", anomalNumber)
print("Number of total anomaly data ", r2lLen)
print("anomally detection correction for r2l data  percenteges is  ", correct)


('Average cost : ', 31.198680513608863)
('Testing for r2l data ', 33, ' anomal data detected ')
('Number of detected Anomaly ', 33)
('Number of total anomaly data ', 36)
('anomally detection correction for r2l data  percenteges is  ', 94.28571428571429)
